In [ ]:
import csv
import os
import re
import pymongo
import urllib.parse   
import string

myclient = pymongo.MongoClient(os.environ["MONGO_STRING"])["extractData"]

In [ ]:
brazilian_letters=string.ascii_letters+string.punctuation+"0123456789ãâáàẽêéèĩîíìõôóòũûúùçÃÂÁÀẼÊÉÈĨÎÍÌÕÔÓÒŨÛÚÙÇ"

def commonReplace(all_data):
    
    all_data=[re.sub("[^ "+brazilian_letters+"]+","",x) for x in all_data]
    
    all_data=[re.sub("http[^ ]+","", x) for x in all_data]
    all_data=[re.sub("^RT[^:]+:","", x) for x in all_data]
    all_data=[x.strip()+'\n' for x in all_data if len(x) > 1]
    
    return set(all_data)

In [ ]:
def twitterText():
    
    result = myclient['twitter'].aggregate([
        {
            '$match': {
                'users': {
                    '$size': 0
                }, 
                'full_text': {
                    '$not': {
                        '$regex': re.compile(r"Just posted a photo")
                    }
                }
            }
        }, {
            '$sort': {
                'timestamp': -1
            }
        }, {
            '$project': {
                'full_text': 1, 
                '_id': 0
            }
        }
    ])
    all_data=[x['full_text'] for x in result]
    return commonReplace(all_data)

In [ ]:
def instagramText():
    result = myclient['instagram'].aggregate([
        {
            '$match': {
                'text': {
                    '$exists': True
                },
  
                'sender_name':"Mariano Aloi"
            }
        }, {
            '$sort': {
                'timestamp': -1
            }
        }, {
            '$project': {
                'text': 1, 
                '_id': 0
            }
        }
    ])
    all_data=[x['text'] for x in result]
    all_data=[x for x in all_data if not (re.match("^Curtiu um",x) or re.match("^Reagiu com",x) or re.match("attachment.$",x) )]
    return commonReplace(all_data)

In [ ]:
def facebookText():
    result = myclient['facebook'].aggregate([
        {
            '$match': {
                'description': {
                    '$exists': True
                },
  
                'sender_name':"Mariano Aloi"
            }
        }, {
            '$sort': {
                'timestamp': -1
            }
        }, {
            '$project': {
                '_id': 0, 
                'description': 1
            }
        }
    ])
    all_data=[x['description'] for x in result]
    return commonReplace(all_data)

In [ ]:
def gmailText   ():
    result = myclient['gmail'].aggregate([
            {
                '$unwind': {
                    'path': '$text'
                }
            }, {
                '$match': {
                    'text.content_type': 'text/plain'
                }
            }, {
                '$project': {
                    '_id': 0, 
                    'text': '$text.msg_text'
                }
            }
        ])
    all_data=[x['text'] for x in result]
    all_data=[re.sub("=(A-Z0-9{2})","%\g<1>",text) for text in all_data]
    all_data=[text.replace("=\n"," ") for text in all_data]
    all_data=[urllib.parse.unquote(text) for text in all_data]
    return commonReplace(all_data)

In [ ]:
with open(os.path.join(os.environ["GOOGLE_DRIVE"],"COLAB","msg.csv"), 'w',  encoding='utf-8') as csvFile:   
    

    csvFile.writelines( twitterText())
    csvFile.writelines( instagramText())
    csvFile.writelines( facebookText())
    # csvFile.writelines( gmailText()) # I tried but the text is very polluted with reply into the message